In [ ]:
# check GPU, colab sometimes doesn't give you any GPU...
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# install h5py, since the original h5py on colab doesn't work
pip install 'h5py==2.10.0' --force-reinstall

In [ ]:
# download data
!wget -r -np -nH --reject "index.html*" --cut-dirs 6 \
 https://krishna.gs.washington.edu/content/members/vagar/Xpresso/data/datasets/pM10Kb_1KTest/

In [ ]:
# imports
%tensorflow_version 1.x

from google.colab import drive
drive.mount('/content/drive')


import pandas as pd
import numpy as np
import tensorflow as tf

import dill

import matplotlib.pyplot as plt

from math import *
import keras
from keras import datasets, layers, models, Model,regularizers, initializers
from keras.models import Model, load_model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adam, SGD
from keras.constraints import maxnorm, nonneg
from keras.activations import sigmoid
from keras.layers import *
from keras.metrics import *
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from hyperopt import hp, STATUS_OK

In [ ]:
# load data
import h5py
import os
datadir="pM10Kb_1KTest"
trainfile = h5py.File(os.path.join(datadir, 'train.h5'), 'r')
X_trainhalflife, X_trainpromoter, y_train, geneName_train = trainfile['data'], trainfile['promoter'], trainfile['label'], trainfile['geneName']
validfile = h5py.File(os.path.join(datadir, 'valid.h5'), 'r')
X_validhalflife, X_validpromoter, y_valid, geneName_valid = validfile['data'], validfile['promoter'], validfile['label'], validfile['geneName']
testfile = h5py.File(os.path.join(datadir, 'test.h5'), 'r')
X_testhalflife, X_testpromoter, y_test, geneName_test = testfile['data'], testfile['promoter'], testfile['label'], testfile['geneName']

leftend = 3000
rightend = 13500

leftpos = 9800
rightpos = 10200

XTrain = X_trainpromoter[:,leftpos:rightpos,:]
XVal = X_validpromoter[:,leftpos:rightpos,:]
XTest = X_testpromoter[:,leftpos:rightpos,:]

YTrain = y_train
YVal = y_valid
YTest = y_test

In [ ]:
# generate data divisions
import random
import os
import pickle

num_of_data_division = 10

train_num = len(XTrain)
val_num = len(XVal)
test_num = len(XTest)

def genAndSaveDivisionLists(train_num, val_num, test_num):
    total_num = train_num + val_num + test_num
    total_set = range(0, total_num)
    train_set = random.sample(total_set, train_num)
    total_set = set(total_set) - set(train_set)
    val_set = random.sample(total_set, val_num)
    test_set = list(set(total_set) - set(val_set))
    return {'train': train_set, 'valid': val_set, 'test': test_set}

for i in range(0, num_of_data_division):
    folder_name = "/Your_Folder/run_" + str(i) + "/"
    os.mkdir(folder_name)
    division_dict = genAndSaveDivisionLists(train_num, val_num, test_num)
    with open(folder_name + "division_dict","wb") as f:
        pickle.dump(division_dict, f)



In [ ]:
# load a specific data division
import pickle

total_x = np.concatenate([XTrain, XVal, XTest], axis=0)
total_y = np.concatenate([YTrain, YVal, YTest], axis=0)
total_gn = np.concatenate([geneName_train, geneName_valid, geneName_test], axis=0)

division_set_num = 0

folder_name = "/Your_Folder/run_" + str(division_set_num) + "/"

with open(folder_name + "division_dict_" + str(division_set_num),"rb") as f:
    division_dict = pickle.load(f)

XTrain = total_x[division_dict['train'],:,:]
XVal = total_x[division_dict['valid'],:,:]
XTest = total_x[division_dict['test'],:,:]
YTrain = total_y[division_dict['train']]
YVal = total_y[division_dict['valid']]
YTest = total_y[division_dict['test']]

print(XTrain.shape)
print(XVal.shape)
print(XTest.shape)
print(YTrain.shape)
print(YVal.shape)
print(YTest.shape)


In [ ]:
# model
def positiveAndSmallRegularizer(x):
  return 1e-1 * (0 * keras.backend.mean(tf.square(x)) - keras.backend.mean(x))

def strong_sigmoid(x):
  return keras.backend.sigmoid(4 * x)

class BiasLayer(Layer):
  def __init__(self, *args, **kwargs):
    super(BiasLayer, self).__init__(*args, **kwargs)

  def build(self, input_shape):
    self.bias = self.add_weight('bias',
                                    shape=input_shape[1:],
                                    initializer=initializers.Constant(0),
                                    trainable=True)
  def call(self, x):
    return x + self.bias

num_of_motifs = 64
motif_by_num = 8

layer_0 = Input(shape=XTrain.shape[1:],name='Input')

layer_1_fun = Conv1D(num_of_motifs, motif_by_num, padding='valid', kernel_initializer='glorot_normal', kernel_constraint=nonneg(), activity_regularizer=regularizers.l1(0.00000002), name='Conv1D_1', bias=False)
layer_1 = layer_1_fun(layer_0)

layer_1_over_bp_num = Lambda(lambda x: x, name='layer_1_over_bp_num')(layer_1)

motif_exists = Dense(num_of_motifs, kernel_initializer=initializers.Identity(), trainable=False, activity_regularizer=regularizers.l1(0), name='motif_exists')(layer_1)
real_motif_counts = Lambda(lambda x: keras.backend.sum(x, axis=1), name='real_motif_counts')(motif_exists)
motif_counts = Lambda(lambda x: keras.backend.mean(x, axis=1), name='motif_counts')(motif_exists)

layer_2_pre = ReLU(name='ReLU_0')(layer_1)
layer_2 = MaxPool1D(50,name='MaxPool_1')(layer_2_pre)

layer_3 = Conv1D(16,7,padding='same', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0), activation='relu',name='Conv1D_2')(layer_2)
layer_4 = MaxPool1D(2,name='MaxPool_2')(layer_3)

layer_5 = Conv1D(8,7,padding='same', kernel_initializer='glorot_normal', kernel_regularizer=regularizers.l2(0), activation='relu',name='Conv1D_3')(layer_4)
layer_6 = MaxPool1D(2,name='MaxPool_3')(layer_5)

layer_7 = Flatten(name='Flatten_1')(layer_6)

layer_8 = Dense(64,name='Dense_1', kernel_regularizer=regularizers.l2(0.0001))(layer_7)
layer_9 = ReLU(name='ReLU_1')(layer_8)
layer_10 = Dropout(0.2,name='dropout_1')(layer_9)

layer_11 = Dense(64,name='Dense_2', kernel_regularizer=regularizers.l2(0.0001))(layer_10)
layer_12 = ReLU(name='ReLU_2')(layer_11)
layer_13 = Dropout(0.2,name='dropout_2')(layer_12)

layer_14 = Dense(num_of_motifs, name='Contextual_Weight', activity_regularizer=regularizers.l1(0.0001))(layer_13)

layer_15 = Multiply(name='Multiply')([motif_counts,layer_14])

layer_16 = Lambda(lambda x: keras.backend.sum(x, axis=1), name='Sum')(layer_15)
layer_17 = BiasLayer(name='Bias')(layer_16)
layer_18 = Lambda(lambda x: keras.backend.expand_dims(x, axis=1), name='Output')(layer_17)

model = Model(inputs=layer_0,outputs=layer_18)
model.summary()
keras.utils.plot_model(model, "model_with_shape_info.png", show_shapes=True)

In [ ]:
# training
model.compile(SGD(lr=0.0005, momentum=0.9),'mean_squared_error', metrics=['mean_squared_error'])
earlystop_cb = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
check_cb = ModelCheckpoint(folder_name + 'bestparams_200_with_b_64_filters.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
result = model.fit(XTrain, YTrain, batch_size=128, shuffle="batch", epochs=100,
        validation_data=[XVal, YVal], callbacks=[earlystop_cb, check_cb])
mse_history = result.history['val_mean_squared_error']

In [ ]:
# calculate accuracy
best_file = folder_name + 'bestparams_200_with_b_64_filters.h5'
model = models.load_model(best_file, custom_objects={'keras': keras, 'tf': tf, 'strong_sigmoid': strong_sigmoid, 'positiveAndSmallRegularizer': positiveAndSmallRegularizer,'motif_by_num':motif_by_num, 'BiasLayer':BiasLayer})
YTrain_Pred = model.predict(XTrain)
print('Correlation of training dataset: '+str(np.corrcoef(YTrain[:].transpose(),YTrain_Pred.transpose())[0,1]))
YVal_Pred = model.predict(XVal)
print('Correlation of validation dataset: '+str(np.corrcoef(YVal[:].transpose(),YVal_Pred.transpose())[0,1]))
YTest_Pred = model.predict(XTest)
print('Correlation of test dataset: '+str(np.corrcoef(YTest[:].transpose(),YTest_Pred.transpose())[0,1]))

In [ ]:
# collect motifs from all runs
import numpy as np

total_x = np.concatenate([XTrain, XVal, XTest], axis=0)
total_y = np.concatenate([YTrain, YVal, YTest], axis=0)
total_gn = np.concatenate([geneName_train, geneName_valid, geneName_test], axis=0)

all_motifs = []
for division_set_num in range(0, 10):
    folder_name = "/Your_Folder/run_" + str(division_set_num) + "/"
    best_file = folder_name + 'bestparams_200_with_b_64_filters.h5'
    model = models.load_model(best_file, custom_objects={'keras': keras, 'tf': tf, 'strong_sigmoid': strong_sigmoid, 'positiveAndSmallRegularizer': positiveAndSmallRegularizer,'motif_by_num':motif_by_num, 'BiasLayer':BiasLayer})
    motif_out = keras.Model(inputs=model.input, outputs=model.get_layer('motif_counts').output)
    NN_found_motifs = motif_out.predict(total_x)
    all_motifs.append(NN_found_motifs)

all_motifs = np.concatenate(all_motifs, axis=1)
    

In [ ]:
# load JASPAR motifs
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from scipy.stats import pearsonr
import pickle
import dill
import numpy as np
import pandas as pd

CR_motif_name = []
for i in range(0, 10):
    for j in range(0, 64):
        CR_motif_name.append("run_" + str(i) + "_motif_" + str(j))

JASPAR_fimo_table = pd.read_csv("/Your_Folder/all_JASPAR_fimo.csv", index_col=0)
JASPAR_fimo = JASPAR_fimo_table.values
JASPAR_fimo_name = JASPAR_fimo_table.columns.values

all_motifs_with_JASPAR = np.concatenate([all_motifs, JASPAR_fimo], axis=1)
all_motif_name = []
all_motif_name.extend(CR_motif_name)
all_motif_name.extend(JASPAR_fimo_name)

scaler = StandardScaler()
scaler.fit(all_motifs_with_JASPAR)
all_motifs_with_JASPAR_std = scaler.transform(all_motifs_with_JASPAR)

print(all_motif_name)


In [ ]:
# feature selection with entropy
from sklearn.feature_selection import RFE
from sklearn.feature_selection import mutual_info_regression
from sklearn.tree import DecisionTreeRegressor
fs = RFE(estimator=DecisionTreeRegressor(), n_features_to_select=300, step=100, verbose=1)
fs.fit(all_motifs_with_JASPAR_std, total_y)

In [ ]:
# get selected features from feature selection
information_selected_i = np.where(fs.support_)[0]
information_score = fs.estimator_.feature_importances_

In [ ]:
# prediction accuracy check with selected
selected_i = set(information_selected_i)
selected_i = sorted(selected_i)
print(selected_i)
selected_motifs_with_JASPAR_std = all_motifs_with_JASPAR_std[:, selected_i]
print(selected_motifs_with_JASPAR_std.shape)

lr = LinearRegression()
lr.fit(selected_motifs_with_JASPAR_std, total_y)
print(lr.score(selected_motifs_with_JASPAR_std, total_y))